In [16]:
# add autoreload magic
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Global parameters
feature_view_name = 'ohlc_feature_view'
feature_view_version = 1
ohlc_window_sec = 60
product_id = 'BTC/USD'
last_n_days_to_fetch_from_store = 90
last_n_days_to_test_model = 7
discretization_thresholds = [-0.0001, 0.0001]
prediction_window_sec = 60*5

In [ ]:
import os
os.environ['HOPSWORKS_API_KEY'] = 'API_KEY'
os.environ['HOPSWORKS_PROJECT_NAME'] = 'project_name'

In [4]:
import pandas as pd
from loguru import logger
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report   

In [5]:
# Step 1    
# Fetch the data from the feature store
from tools.ohlc_data_reader import OhlcDataReader

ohlc_data_reader = OhlcDataReader(
    ohlc_window_sec=ohlc_window_sec,
    feature_view_name=feature_view_name,
    feature_view_version=feature_view_version,
)

logger.info('Fetching OHLC data from the feature store')

ohlc_data = ohlc_data_reader.read_from_offline_store(
    product_id=product_id,
    last_n_days=last_n_days_to_fetch_from_store,
)

c:\src\real-time-ml-system\services\price_predictor\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-26 19:25:15,904 INFO: Initializing external client
2025-04-26 19:25:15,905 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-26 19:25:22,588 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1174676


2025-04-26 19:25:25.230 | INFO     | __main__:<module>:11 - Fetching OHLC data from the feature store


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.38s) 


In [6]:
ohlc_data

,timestamp,open,high,low,close,product_id
0,1711330020000,66684.9,66699.0,66684.9,66699.0,BTC/USD
1,1711330080000,66699.0,66699.0,66699.0,66699.0,BTC/USD
2,1711330140000,66699.0,66699.0,66699.0,66699.0,BTC/USD
3,1711330200000,66699.0,66699.0,66699.0,66699.0,BTC/USD
4,1711330260000,66699.0,66745.0,66699.0,66739.4,BTC/USD
...,...,...,...,...,...,...
126148,1719093960000,64281.0,64281.0,64281.0,64281.0,BTC/USD
126149,1719094020000,64281.0,64281.0,64281.0,64281.0,BTC/USD
126150,1719094680000,64257.9,64258.0,64257.9,64258.0,BTC/USD
126151,1719094740000,64257.9,64257.9,64257.9,64257.9,BTC/USD


In [7]:
# add a column to ohlc_data with a human-readable data, using
# the ohlc_data['timestamp'] column in milliseconds
ohlc_data['datetime'] = pd.to_datetime(ohlc_data['timestamp'], unit='ms')

In [8]:
from src.training import split_train_test

# Step 2
# Split the data into training and testing using a cutoff date
logger.info('Splitting the data into training and testing')
ohlc_train, ohlc_test = split_train_test(
    ohlc_data=ohlc_data,
    last_n_days_to_test_model=last_n_days_to_test_model,
)

# print(ohlc_train.head())
# print(ohlc_test.head())

2025-04-26 19:25:48.069 | INFO     | __main__:<module>:5 - Splitting the data into training and testing


In [9]:
from src.training import interpolate_missing_candles

# Step 3
# Preprocess the data for training and for testing
# Interpolate missing candles
logger.info('Interpolating missing candles for training data')
ohlc_train = interpolate_missing_candles(ohlc_train, ohlc_window_sec)
logger.info('Interpolating missing candles for testing data')
ohlc_test = interpolate_missing_candles(ohlc_test, ohlc_window_sec)

2025-04-26 19:25:48.227 | INFO     | __main__:<module>:6 - Interpolating missing candles for training data
2025-04-26 19:25:48.280 | INFO     | __main__:<module>:8 - Interpolating missing candles for testing data


In [10]:
from src.training import create_target_metric

# Step 4
# Create the target metric as a new column in our dataframe for training and testing
logger.info('Creating the target metric')
ohlc_train = create_target_metric(
    ohlc_train,
    ohlc_window_sec,
    discretization_thresholds,
    prediction_window_sec,
)
ohlc_test = create_target_metric(
    ohlc_test,
    ohlc_window_sec,
    discretization_thresholds,
    prediction_window_sec,
)

2025-04-26 19:25:48.422 | INFO     | __main__:<module>:5 - Creating the target metric


In [11]:
# Plot distribution of the target
logger.info('Distribution of the target in the training data')
logger.debug(ohlc_train['target'].value_counts())
logger.info('Distribution of the target in the testing data')
logger.debug(ohlc_test['target'].value_counts())

2025-04-26 19:25:48.633 | INFO     | __main__:<module>:2 - Distribution of the target in the training data
2025-04-26 19:25:48.650 | DEBUG    | __main__:<module>:3 - target
2.0    50572
0.0    48222
1.0    20541
Name: count, dtype: int64
2025-04-26 19:25:48.650 | INFO     | __main__:<module>:4 - Distribution of the target in the testing data
2025-04-26 19:25:48.650 | DEBUG    | __main__:<module>:5 - target
0.0    3519
2.0    3507
1.0    3049
Name: count, dtype: int64


In [12]:
# Before training, let's split the features and the target
X_train = ohlc_train.drop(columns=['target'])
y_train = ohlc_train['target']
X_test = ohlc_test.drop(columns=['target'])
y_test = ohlc_test['target']

In [13]:
from src.baseline_model import BaselineModel

# create model
model = BaselineModel(
    n_candles_into_future=prediction_window_sec // ohlc_window_sec,
    discretization_thresholds=discretization_thresholds,
)

# generate predictions
y_test_predictions = model.predict(X_test)

# evalute our dummy model
# Let's evaluate the model. It is a classifier with 3 classes

print('****** TEST DATA ******')
# Compute accuracy using scikit-learn
accuracy = accuracy_score(y_test, y_test_predictions)
print(f'Accuracy of the model on test data: {accuracy}')

print(f'Classification report of the model:')
print(classification_report(y_test, y_test_predictions))

# generate predictions
print('****** TRAINING DATA ******')
y_train_predictions = model.predict(X_train)
accuracy = accuracy_score(y_train, y_train_predictions)
print(f'Accuracy of the model: {accuracy}')

print(f'Classification report of the model:')
print(classification_report(y_train, y_train_predictions))

****** TEST DATA ******
Accuracy of the model on test data: 0.5105707196029776
Classification report of the model:
              precision    recall  f1-score   support

         0.0       0.46      0.46      0.46      3519
         1.0       0.63      0.63      0.63      3049
         2.0       0.46      0.46      0.46      3507

    accuracy                           0.51     10075
   macro avg       0.52      0.52      0.52     10075
weighted avg       0.51      0.51      0.51     10075

****** TRAINING DATA ******
Accuracy of the model: 0.4291197050320526
Classification report of the model:
              precision    recall  f1-score   support

         0.0       0.44      0.44      0.44     48222
         1.0       0.33      0.33      0.33     20541
         2.0       0.46      0.46      0.46     50572

    accuracy                           0.43    119335
   macro avg       0.41      0.41      0.41    119335
weighted avg       0.43      0.43      0.43    119335



In [14]:
from src.feature_engineering import (add_momentum_indicators,
add_volatility_indicators)
X_train = add_momentum_indicators(X_train)
X_train = add_volatility_indicators(X_train)
X_train

,timestamp,open,high,low,close,product_id,datetime,rsi,momentum,std
0,1711330020000,66684.9,66699.0,66684.9,66699.0,BTC/USD,2024-03-25 01:27:00,NaN,NaN,NaN
1,1711330080000,66699.0,66699.0,66699.0,66699.0,BTC/USD,2024-03-25 01:28:00,NaN,NaN,NaN
2,1711330140000,66699.0,66699.0,66699.0,66699.0,BTC/USD,2024-03-25 01:29:00,NaN,NaN,NaN
3,1711330200000,66699.0,66699.0,66699.0,66699.0,BTC/USD,2024-03-25 01:30:00,NaN,NaN,NaN
4,1711330260000,66699.0,66745.0,66699.0,66739.4,BTC/USD,2024-03-25 01:31:00,NaN,NaN,16.160000
...,...,...,...,...,...,...,...,...,...,...
119330,1718489820000,66042.0,66042.0,66035.1,66035.1,BTC/USD,2024-06-15 22:17:00,10.958241,-19.8,8.333791
119331,1718489880000,66035.2,66035.2,66035.2,66035.2,BTC/USD,2024-06-15 22:18:00,11.341698,-19.7,8.956744
119332,1718489940000,66035.1,66053.8,66035.1,66047.9,BTC/USD,2024-06-15 22:19:00,44.204812,-7.0,7.642779
119333,1718490000000,66047.9,66047.9,66047.9,66047.9,BTC/USD,2024-06-15 22:20:00,44.204812,-7.0,5.706844


In [17]:
from src.feature_engineering import add_features
X_train = add_features(X_train)
X_train

,timestamp,open,high,low,close,product_id,datetime,rsi,momentum,std
0,1711330020000,66684.9,66699.0,66684.9,66699.0,BTC/USD,2024-03-25 01:27:00,NaN,NaN,NaN
1,1711330080000,66699.0,66699.0,66699.0,66699.0,BTC/USD,2024-03-25 01:28:00,NaN,NaN,NaN
2,1711330140000,66699.0,66699.0,66699.0,66699.0,BTC/USD,2024-03-25 01:29:00,NaN,NaN,NaN
3,1711330200000,66699.0,66699.0,66699.0,66699.0,BTC/USD,2024-03-25 01:30:00,NaN,NaN,NaN
4,1711330260000,66699.0,66745.0,66699.0,66739.4,BTC/USD,2024-03-25 01:31:00,NaN,NaN,16.160000
...,...,...,...,...,...,...,...,...,...,...
119330,1718489820000,66042.0,66042.0,66035.1,66035.1,BTC/USD,2024-06-15 22:17:00,10.958241,-19.8,8.333791
119331,1718489880000,66035.2,66035.2,66035.2,66035.2,BTC/USD,2024-06-15 22:18:00,11.341698,-19.7,8.956744
119332,1718489940000,66035.1,66053.8,66035.1,66047.9,BTC/USD,2024-06-15 22:19:00,44.204812,-7.0,7.642779
119333,1718490000000,66047.9,66047.9,66047.9,66047.9,BTC/USD,2024-06-15 22:20:00,44.204812,-7.0,5.706844
